<a href="https://colab.research.google.com/github/georgeaidinis/NTUA/blob/master/NN/Lab%201/Lab_1_Small_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Αϊδίνης Γιώργος 03116031

Κολιός Παναγιώτης 03116100

---

Ομάδα M.B.8

---


#Lab 1: Επιβλεπόμενη Μάθηση - Ταξινόμηση - Μικρό Dataset (S11 - Quality Assessment of Digital Colposcopies)

Αρχικά ενημερώνουμε τις βιβλιοθήκες που θα χρησιμοποιήσουμε.



In [5]:
!pip install --upgrade pip #upgrade pip package installer
!pip install scikit-learn --upgrade #upgrade scikit-learn package
!pip install numpy --upgrade #upgrade numpy package
!pip install pandas --upgrade #--upgrade #upgrade pandas package

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 7.0MB 4.7MB/s 
  Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3


Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.17.4)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (0.25.3)




1.   Το dataset περιγράφει μετρήσεις κολποσκοπήσεων και την κατάσταση (ετικέτα) των αντίστοιχων κόλπων, όπως προκύπτει από τις εκτιμήσεις καθενός από 6 ειδικούς και την εκτίμηση της πλειοψηφίας. Οι μετρήσεις και οι εκτιμήσεις είναι διαφορετικές για τις διαφορετικές μεθόδους με τις οποίες πραγματοποιούνται οι κολποσκοπήσεις: Hinselmann, Green, Schiller.



2.    Έχουμε συνολικά 287 δείγματα, 92 από τη μέθοδο Schiller, 98 από τη μέθοδο Green και 97 από τη μέθοδο Hinselmann. Σε κάθε δείγμα περιέχονται μετρήσεις για 62 χαρακτηριστικά-παρατηρήσεις από τα οποία προέκυψαν οι εκτιμήσεις. Όλα τα χαρακτηριστικά είναι διατεταγμένα.Οι μετρήσεις είναι αριθμητικά δεδομένα και αφορούν τιμές όπως τις επιφάνειες περιοχών του κόπλου.

3.    Υπάρχουν επικεφαλίδες στην πρώτη γραμμή πάνω από τα χαρακτηριστικά και τις ετικέτες, οι οποίες θα πρέπει να αφαιρεθούν. Δεν υπάρχει στήλη για την αρίθμηση των γραμμών.

4.    Η τιμή της κατάστασης μπορεί να πάρει δύο τιμές, 0 για κακή και 1 για καλή. Όπως υποδεικνύεται από τις FAQ το πρόβλημα θα αναλυθεί ως binary classification λαμβάνοντας υπόψην μόνο τις εκτιμήσεις της πλειοψηφίας. Έτσι οι στήλες των ετικετών που αφορούν μεμονωμένα τον κάθε ειδικό αφαιρούνται και μένει μόνο η τελευταία στήλη με τις εκτιμήσεις της πλειοψηφίας. Η στήλη αυτή είναι η τελευταία (θέση 69 στον αρχικό πίνακα). Παρατηρούμε οτι όλες οι ισοψηφίες (3-3) επιλύονται θεωρώντας την κατάσταση ως καλή, γεγονός που ίσως μας αναγκάσει να αυξήσουμε αργότερα το πλήθος των δειγμάτων με ετικέτα 0 ή να διαγράψουμε δείγματα με ετικέτα 1.

    
5.    Συνενώνουμε τα αρχεία που αφορούν τις 3 διαφορετικές μεθόδους. Μετά τη συνένωση οι επικεφαλίδες προστέθηκαν ως γραμμές, με αποτέλεσμα να έχουμε 3 φορές την ίδια γραμμή με τις επικεφαλίδες, τις οποίες και αφαιρούμε.

6.    Δεν υπάρχουν απουσιάζουσες τιμές.



In [0]:
from urllib.request import urlretrieve
import os
import zipfile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")


In [7]:
download("http://archive.ics.uci.edu/ml/machine-learning-databases/00384/Quality%20Assessment%20-%20Digital%20Colposcopy.zip","QADC.zip")
print("All the files are downloaded")

Download file... QADC.zip ...
File downloaded
All the files are downloaded


In [8]:
!ls

QADC.zip  sample_data


In [0]:
!unzip -q "QADC.zip"

In [10]:
!ls

 QADC.zip  'Quality Assessment - Digital Colposcopy'   sample_data


Rename directory for better handling



In [0]:
!mv Quality\ Assessment\ -\ Digital\ Colposcopy QADC

Remove attributes line for the 2nd and 3rd files:

In [0]:
with open("QADC/hinselmann.csv",'r') as f:
    with open("QADC/hinselmann1.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
            f1.write(line)

In [0]:
with open("QADC/schiller.csv",'r') as f:
    with open("QADC/schiller1.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
            f1.write(line)

create one big .csv

In [0]:
!cat QADC/green.csv QADC/hinselmann1.csv QADC/schiller1.csv > all.csv

We needed to concatenate the 3 .csv files && remove the category names from the second and third files gia na min exoume asxeta instances
We also need to remove the attribute of the experts, apart from the concensus

In [0]:
import pandas as pd
import numpy as np

df = pd.read_csv("all.csv")
df = df.drop(df.columns[[62, 63, 64, 65, 66, 67]], axis=1)

Let's print number, names and types of attributes

In [16]:
print ("We have ",len(df.columns), " attributes.")
for i in range(0, len(df.columns)):
    print('{:<10}{:<40}{:<10}{:<20}'.format(str(i+1), str(df.columns[i]),"type: ", str(df.dtypes[df.columns[i]])))


We have  63  attributes.
1         cervix_area                             type:     float64             
2         os_area                                 type:     float64             
3         walls_area                              type:     float64             
4         speculum_area                           type:     float64             
5         artifacts_area                          type:     float64             
6         cervix_artifacts_area                   type:     float64             
7         os_artifacts_area                       type:     float64             
8         walls_artifacts_area                    type:     float64             
9         speculum_artifacts_area                 type:     float64             
10        cervix_specularities_area               type:     float64             
11        os_specularities_area                   type:     float64             
12        walls_specularities_area                type:     float64             
13 

We need to count the occurences of each label (healthy or unhealthy):

In [17]:
labels_df = df.iloc[:, [62]]
labels = labels_df.values.reshape(287,)
positive = 0
negative = 0
for i in range(0, len(labels)):
    positive += labels[i]==1
    negative += labels[i]==0
print("positive = ", positive, " negative: ", negative)

positive =  216  negative:  71


Βλεπουμε οτι εχουμε unbalanced data set, καθως η κλαση των positive(υγειων) ειναι κατα πολυ μεγαλυτερη σε μεγεθος απο την κλαση των negative. (75-25)

Train - Test split

In [18]:
from sklearn.model_selection import train_test_split


train, test, train_labels, test_labels = train_test_split(df, labels, test_size=0.2)
print(len(train))


229


In [19]:
from scipy import stats as st


# standardization
std_x = st.zscore(train)
Xvar = np.var(std_x, axis=0)
print(Xvar)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


Undersampling - Oversampling

In [20]:
positive = 0
negative = 0
for i in range(0, len(train_labels)):
    positive += train_labels[i]==1
    negative += train_labels[i]==0
print("positive = ", positive, " negative: ", negative)

positive =  168  negative:  61


In [0]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(sampling_strategy=0.65)
train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)


In [22]:
positive = 0
negative = 0
for i in range(0, len(train_labels_oversampled)):
    positive += train_labels_oversampled[i]==1
    negative += train_labels_oversampled[i]==0
print("positive = ", positive, " negative: ", negative)

positive =  168  negative:  109


In [0]:
from imblearn.under_sampling import RandomUnderSampler

ros = RandomUnderSampler(random_state=0)
train_undersampled, train_labels_undersampled = ros.fit_sample(train_oversampled,train_labels_oversampled)

In [24]:
positive = 0
negative = 0
for i in range(0, len(train_labels_undersampled)):
    positive += train_labels_undersampled[i]==1
    negative += train_labels_undersampled[i]==0
print("positive = ", positive, " negative: ", negative)

positive =  109  negative:  109


In [25]:
Xvar = np.var(train_undersampled, axis=0)
print(Xvar)

[6.12691261e-02 3.61629869e-05 5.06633006e-02 3.31683580e-02
 5.97877841e-03 4.17111071e-03 2.52883102e-02 1.32303302e-02
 1.62743613e-02 3.79363879e-04 1.84239646e-03 5.22005865e-04
 1.07333419e-02 1.40059234e-03 1.91484385e-02 2.40058327e+03
 2.83609049e+02 1.48765018e+03 3.88073445e+03 2.67541489e+03
 2.81423502e+02 1.72347159e+03 4.19020519e+03 4.29400117e+03
 4.53625821e+02 2.55803397e+03 6.93722001e+03 1.36194867e+03
 4.85024891e+02 6.34514587e+02 3.05943253e+03 1.22467463e+03
 4.75230701e+02 6.31961675e+02 2.76784898e+03 2.63780393e+03
 6.17556154e+02 1.23128402e+03 5.27943615e+03 5.02250143e-01
 4.09780081e-01 3.26893621e+03 4.72034553e+02 4.08366776e+03
 4.03883168e+02 4.18070963e-01 6.08136759e-02 3.12170936e+03
 3.82062317e+02 2.01005609e+03 4.81268714e+02 4.15469100e-02
 5.97261229e-02 3.76501587e-02 6.58716640e-02 2.25062652e-02
 1.98287332e-01 4.41239669e-02 5.43453165e-02 9.74883079e+03
 7.31094441e-02 3.23395859e-02 2.50000000e-01]


In [26]:
from scipy import stats as st
from sklearn import preprocessing


train_scaled = preprocessing.scale(train_undersampled)

Xvar = np.var(train_scaled, axis=0)
print(Xvar)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [27]:
Yvar = np.var(np.asarray(test), axis=0)
print(Yvar)

[5.01536118e-02 4.41991898e-05 2.81535435e-02 3.13689254e-02
 5.72414548e-03 2.33192761e-03 2.17612781e-02 7.98061016e-03
 1.66159378e-02 1.34389384e-03 4.58203757e-03 5.88747471e-04
 1.13288016e-02 1.00843653e-03 1.68077244e-02 2.19638544e+03
 2.11328095e+02 1.67776569e+03 3.13766138e+03 2.29478853e+03
 2.40123287e+02 1.62062062e+03 3.44920301e+03 3.40560817e+03
 3.85452344e+02 2.31832386e+03 5.26379716e+03 1.04808799e+03
 3.11456889e+02 6.78684267e+02 2.04040549e+03 9.09464542e+02
 3.26039380e+02 6.15038098e+02 1.85596975e+03 1.94031903e+03
 4.31011151e+02 9.51666927e+02 3.79099343e+03 4.51687200e-01
 2.02965863e-01 2.64748970e+03 3.09842192e+02 2.36892387e+03
 2.63756651e+02 3.81609055e-01 5.62918344e-02 2.67513543e+03
 4.13497564e+02 6.84060310e+02 2.51571463e+02 1.77867905e-02
 4.91124673e-02 1.89743576e-02 5.41145530e-02 1.27377657e-02
 1.68569381e-01 1.85292408e-02 4.49743714e-02 1.07219739e+04
 4.98401337e-02 2.20424377e-02 1.42687277e-01]


In [28]:
scaler = preprocessing.StandardScaler().fit(train_undersampled)
test_scaled = scaler.transform(test)
Yvar = np.var(np.asarray(test_scaled), axis=0)
print(Yvar)

[0.81857887 1.22222177 0.55569896 0.94574852 0.95741054 0.55906634
 0.86052718 0.60320567 1.02098863 3.54249288 2.48699869 1.12785605
 1.05547756 0.72000717 0.87775953 0.91493824 0.74513876 1.12779584
 0.80852257 0.85773184 0.85324533 0.94032337 0.8231585  0.79310835
 0.84971429 0.90629127 0.75877616 0.76955029 0.6421462  1.06961177
 0.66692286 0.74261728 0.68606548 0.97322056 0.67054589 0.73558122
 0.6979303  0.77290609 0.71806786 0.89932717 0.49530436 0.80989335
 0.65639727 0.58009711 0.65305185 0.91278536 0.92564433 0.8569457
 1.0822778  0.34031902 0.52272557 0.42811344 0.82229458 0.50396488
 0.82151489 0.56596533 0.85012683 0.41993597 0.82756665 1.09982152
 0.6817195  0.68159307 0.57074911]


In [0]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from scipy import stats as st
from sklearn import neighbors
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier


"""
    This is a function that applies a transformation (selection) on a dataset, 
    based on the type of the selector. Takes 3 arguments, one is mandatory and 
    is the dataset, which is a tuple of data. It needs to be in the form:
    (train_data, train_labels, test_data, test_labels). The other two can be
    None, and when they are, no transformation happens. The selector argument
    is a string and can be one of two things: either "PCA" or 
    "Variance Threshold". In either case, there can be arguments, and in "PCA"
    the arguments can be an int of the number of principal components, and in
    "Variance Threshold", it can be a float. Returns the transformed dataset.
"""
def Apply_selector(data, selector = None, arguments = None):
    (train, train_labels, test, test_labels) = data
    if selector == "PCA":    
        if arguments:
            number_of_components = arguments
            pca = PCA(n_components = number_of_components)
            pca.fit(train)
            train_reduced = pca.transform(train)
            test_reduced = pca.transform(test)
        else:
            pca = PCA()
            pca.fit(train)
            train_reduced = pca.transform(train)
            test_reduced = pca.transform(test)
    elif selector == "Variance Threshold":
        if arguments:
            t = arguments
            sel = VarianceThreshold(threshold=t)
            train_reduced = sel.fit_transform(train)
            test_reduced = sel.transform(test)
        else:
            sel = VarianceThreshold()
            train_reduced = sel.fit_transform(train)
            test_reduced = sel.transform(test)
    else:
        (train_reduced, train_labels, test_reduced, test_labels) = data
    return (train_reduced, train_labels, test_reduced, test_labels)


"""
    This is a function that applies resampling on a dataset, based on the 
    type of sampling method. Takes 3 arguments, one is mandatory and 
    is the dataset, which is a tuple of data. It needs to be in the form:
    (train_data, train_labels, test_data, test_labels). The other two can be
    None, and when they are, no sampling happens. The sampling argument
    is a string and can be one of three things: "Over", "Under" or 
    "OverUnder". In any case, there can be arguments, and in Under or Over or
    OverUnder it can be a float with the ratio of samples between classes. 
    Returns the resampled dataset.
"""
def Apply_sampling(data, sampling = None, arguments = None):
    (train, train_labels, test, test_labels) = data
    if sampling == "Over":    
        if arguments:
            ratio = arguments
            ros = RandomOverSampler(sampling_strategy=ratio)
            train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)
        else:
            ros = RandomOverSampler()
            train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)
        return (train_oversampled, train_labels_oversampled, test, test_labels)
    elif sampling == "Under":
        if arguments:
            ratio = arguments
            rus = RandomUnderSampler(sampling_strategy=ratio)
            train_undersampled, train_labels_undersampled = rus.fit_sample(train,train_labels)
        else:
            rus = RandomUnderSampler()
            train_undersampled, train_labels_undersampled = rus.fit_sample(train,train_labels)
        return (train_undersampled, train_labels_undersampled, test, test_labels)
    elif sampling=="OverUnder":
        if arguments:    
            ratio = arguments
            ros = RandomOverSampler(sampling_strategy=ratio)
            train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)
            rus = RandomUnderSampler()
            train_undersampled, train_labels_undersampled = rus.fit_sample(train_oversampled,train_labels_oversampled)
        else:
            ros = RandomOverSampler()
            train_oversampled, train_labels_oversampled = ros.fit_sample(train,train_labels)
            rus = RandomUnderSampler()
            train_undersampled, train_labels_undersampled = rus.fit_sample(train_oversampled,train_labels_oversampled)
        return (train_undersampled, train_labels_undersampled, test, test_labels)
    else:
        return data



"""
    This is a function that applies a transformation (standardization) on a dataset, 
    based on the type of the standardizer. Takes 3 arguments, one is mandatory and 
    is the dataset, which is a tuple of data. It needs to be in the form:
    (train_data, train_labels, test_data, test_labels). The other two can be
    None, and when they are, no transformation happens. The selector argument
    is a string and can be one of two things: either "zscore" or 
    "minmax". Argument arguments is always None. Returns the transformed dataset.
"""
def Apply_standardizer(data, standardizer = None,  arguments = None):
    (train, train_labels, test, test_labels) = data
    if standardizer=="zscore":
        std_train = st.zscore(train)
    elif standardizer=="minmax":
        std_train = (train - np.min(train) )/ (np.max(train) - np.min(train))
    else:
        std_train = train
    return (std_train, train_labels, test, test_labels)

"""
    This is a function that creates a classifier object. It does not train, 
    it takes the name of the classifier, along with the arguments we want 
    to pass to the instance creator for the classifier and returns the 
    classifier object.
"""
def Create_classifier(classifier,  arguments = None):
    if classifier=="kNN":
        if arguments and arguments!=-1:
            clf = neighbors.KNeighborsClassifier(n_jobs = -1, n_neighbors = arguments)
        else:
            clf = neighbors.KNeighborsClassifier(n_jobs = -1)
    else:
        if arguments:
            strat = arguments
            clf = DummyClassifier(strategy=strat)
        else:
            clf = neighbors.KNeighborsClassifier()
    return clf


In [0]:
from sklearn.metrics import accuracy_score


"""
    This is an implementation for the function imblearn.pipeline.Pipeline().
    The way this function works is, it takes as input two arguments. The first
    is a list. This list contains the steps that need to be executed in the 
    pipeline, IN ORDER. Always the last step is the classifier. Each step is
    a tuple, which contains the name of the step, and the arguments needed to 
    execute the step, like so: ("name", arguments). The second argument of the 
    function is a tuple of data. It needs to be in the form:
    (train_data, train_labels, test_data, test_labels). The function outputs a
    tuple, which contains the processed data, ready for fitting the classier, 
    and the classifier object, as specified by the arguments, like so:
    returns: (processed_data, classifier_object)
"""
def Pipeline(steps, data):
    steps_dict = {"selector": Apply_selector,
                  "sampling": Apply_sampling,
                  "standardizer": Apply_standardizer,
                  "kNN": Create_classifier,
                  "dummy": Create_classifier}
    for step in steps:
        if step[0]!="kNN" and step[0]!="dummy":
            data = steps_dict[step[0]](data,step[1])
        else:
            return ( data, Create_classifier(step[0], arguments = step[1]) )


"""
    This is an implementation for the function gridsearchcv. It takes 3
    arguments, a tuple (model_name, model_parameters), the data to be fitted
    and trained, and the specific parameters. The data parameter needs to be 
    in the form: (train_data, train_labels, test_data, test_labels). Lastly, 
    the Steps argument is a list, that has the steps for the data preprocessing,
    as described in the Pipeline Function. For this exercise, we are dealing 
    only with dummy classifiers and kNN. The model_parameters if not None, is 
    a list of parameters (strings for dummy, ints for kNN).
    Returns (classifier_object, best_parameter, accuracy_list)
"""
def model_tuning(model, data, preprocessing_steps = None):
    if model[0]=="dummy":
        if model[1]!=None:
            accuracy_list = []
            best_parameter = model[1][0]
            for parameter in model[1]:
                if preprocessing_steps:
                    steps = preprocessing_steps + [(model[0], parameter)]
                else:
                    steps = [(model[0], parameter)]
                (processed_data, clf) = Pipeline(steps, data)
                (train, train_labels, test, test_labels) = processed_data
                clf = clf.fit(train, train_labels)
                preds = clf.predict(test)
                accuracy_list.append(accuracy_score(test_labels, preds))
                if accuracy_score(test_labels, preds) == max(accuracy_list):
                    best_parameter = parameter
        else:
            accuracy_list = []
            if preprocessing_steps:
                steps = preprocessing_steps + [(model[0], model[1])]
            else:
                steps = [(model[0], model[1])]
            (processed_data, clf) = Pipeline(steps, data)
            (train, train_labels, test, test_labels) = processed_data
            clf = clf.fit(train, train_labels)
            preds = clf.predict(test)
            accuracy_list.append(accuracy_score(test_labels, preds))
            best_parameter = None
    elif model[0] == "kNN":
        if model[1]== -1:
            accuracy_list = []
            if preprocessing_steps:
                steps = preprocessing_steps + [(model[0], model[1])]
            else:
                steps = [(model[0], model[1])]
            (processed_data, clf) = Pipeline(steps, data)
            (train, train_labels, test, test_labels) = processed_data
            clf = clf.fit(train, train_labels)
            preds = clf.predict(test)
            accuracy_list.append(accuracy_score(test_labels, preds))
            best_parameter = None
        else:
            accuracy_list = []
            best_parameter = model[1][0]
            for parameter in model[1]:
                if preprocessing_steps:
                    steps = preprocessing_steps + [(model[0], parameter)]
                else:
                    steps = [(model[0], parameter)]
                (processed_data, clf) = Pipeline(steps, data)
                (train, train_labels, test, test_labels) = processed_data
                clf = clf.fit(train, train_labels)
                preds = clf.predict(test)
                accuracy_list.append(accuracy_score(test_labels, preds))
                if accuracy_score(test_labels, preds) == max(accuracy_list):
                    best_parameter = parameter
    return (clf, best_parameter, accuracy_list )

In [125]:
df = pd.read_csv("all.csv")
df = df.drop(df.columns[[62, 63, 64, 65, 66, 67]], axis=1)
train, test, train_labels, test_labels = train_test_split(df, labels, test_size=0.2)
print(len(train), len(test))
data = (train, train_labels, test,  test_labels)
print("1:")
clf, best_parameter, acc_list = model_tuning( ("kNN", -1),  data,    preprocessing_steps= [("selector", "PCA"),("sampling","Under",0.5), ("standardizer", "zscore")]   )    
print(best_parameter, max(acc_list))
print("2:")
clf, best_parameter, acc_list = model_tuning( ("dummy", None),  data,    preprocessing_steps= [("selector", "PCA"),("sampling","Under",0.5), ("standardizer", "zscore")]   )    
print(best_parameter, max(acc_list))

print("3:")
acc_list = []
data = (train, train_labels, test,  test_labels)
model = ("kNN", [1,2,3,4,5,6,7,8,9,10,20,30,40,50,100])
clf, best_parameter, acc_list = model_tuning(model, data)
print(best_parameter, max(acc_list))


229 58
1:
None 0.603448275862069
2:
None 0.46551724137931033
3:
20 0.8793103448275862


In [126]:
print(acc_list)

[0.8275862068965517, 0.7413793103448276, 0.8448275862068966, 0.8275862068965517, 0.8620689655172413, 0.8448275862068966, 0.8793103448275862, 0.8620689655172413, 0.8448275862068966, 0.8620689655172413, 0.8793103448275862, 0.8448275862068966, 0.8448275862068966, 0.8448275862068966, 0.8448275862068966]
